In [8]:
overpass_api_query = """
[out:json];
area[name="秋田県"]->.a;
(
  nwr[amenity="hospital"](area.a);
);
out geom;
"""

import requests

overpass_api_res = requests.post("https://overpass-api.de/api/interpreter" , data = {"data": overpass_api_query})
overpass_api_json = overpass_api_res.json()

elements = []

for element in overpass_api_json['elements']:
    if 'geometry' in element:
        elements.append(element)

print(len(elements))

32


In [9]:
import psycopg2

# PostgreSQLの情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# テーブルを作成
cursor.execute('CREATE TABLE hospitals_akita_2023_07_18 (id BIGINT, name VARCHAR, name_en VARCHAR, geom GEOMETRY(Polygon, 4326));')

# 公園データを挿入
for element in elements:
    element_id = element['id']
    element_name = element['tags'].get('name', '').replace("'", "''")
    element_name_en = element['tags'].get('name:en', '').replace("'", "''")

    coordinates = element['geometry']

    # Polygonではないelementはスキップする
    if len(coordinates) < 3:
        continue

    # 座標データをPostGISの形式に変換
    polygon = 'POLYGON(('
    for coord in coordinates:
        lon = coord['lon']
        lat = coord['lat']
        polygon += f'{lon} {lat},'
    polygon = polygon.rstrip(',')  # 末尾のカンマを削除
    polygon += '))'

    # テーブルに挿入
    insert_query = f"INSERT INTO hospitals_akita_2023_07_18 (id, name, name_en, geom) VALUES ({element_id}, '{element_name}', '{element_name_en}', ST_SetSRID(ST_GeomFromText('{polygon}'), 4326));"
    print(insert_query)
    cursor.execute(insert_query)

# コミットして変更を確定
conn.commit()

# 接続を閉じる
conn.close()

INSERT INTO hospitals_akita_2023_07_18 (id, name, name_en, geom) VALUES (206374371, '', '', ST_SetSRID(ST_GeomFromText('POLYGON((140.5482711 39.3139934,140.5500074 39.3137018,140.5514063 39.3134346,140.5504926 39.3104012,140.5473591 39.3109664,140.5482711 39.3139934))'), 4326));
INSERT INTO hospitals_akita_2023_07_18 (id, name, name_en, geom) VALUES (206374374, '平鹿総合病院', 'Hiraka General Hospital', ST_SetSRID(ST_GeomFromText('POLYGON((140.5484663 39.3134364,140.5484426 39.3133553,140.5484243 39.3132988,140.5481198 39.3122485,140.5482475 39.3122262,140.5483585 39.3122068,140.5484243 39.3121954,140.5484596 39.3121892,140.5484833 39.3122705,140.5485658 39.3122561,140.5485372 39.3121583,140.5488705 39.3121001,140.5489179 39.3122624,140.5490764 39.3122347,140.5490298 39.3120753,140.5493709 39.3120157,140.5494706 39.3123572,140.5497248 39.3123128,140.549759 39.3124301,140.5494982 39.3124757,140.549525 39.3125674,140.5493288 39.3126017,140.5493332 39.3126166,140.5493962 39.3128324,140.5494747 